# Model plots

In [11]:
import stella
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm

In [2]:
ds = stella.FlareDataSet(fn_dir='/Users/azib/documents/open_source/nets2/models/comets10k-newtails/',catalog='/Users/azib/documents/open_source/nets2/comets10k-newtails.txt',cadences=168)

Reading in training set files.


100%|██████████████████████████████████| 9990/9990 [00:02<00:00, 3589.88it/s]


Number of positive class training data: 7752
Number of negative class training data: 7011
Number of positive class validation data: 969
Number of negative class validation data: 876
Size of augmented data (training set only): 0
Class label (training): 0, Count: 7011
Class label (training): 1, Count: 7752
Class label (validation): 0, Count: 876
Class label (validation): 1, Count: 969
Total size of training set: 14763
Total size of validation set: 1845
Total size of test set: 1846
Approximate class imbalance: -11.0


In [12]:
ind_pc = np.where(ds.train_labels == 1)[0]

In [15]:
# os.makedirs('exocomets-newtails/',exist_ok=True)
# dsmodels = ds.train_data[ind_pc]
# num_sets = dsmodels.shape[0] // 100

# for set_index in tqdm(range(num_sets)):
#     start_index = set_index * 100
#     end_index = min((set_index + 1) * 100, dsmodels.shape[0])  # Ensure not to exceed the length of the data

#     # Create a new 10x10 grid of subplots for each set
#     fig, axs = plt.subplots(10, 10, figsize=(20, 20))

#     # Flatten the axes array for easier iteration
#     axs = axs.flatten()

#     # Iterate over the elements in the current set and plot each in the grid
#     for i in range(start_index, end_index):
#         plot_index = i % 100  # Calculate the index within the current grid
#         axs[plot_index].plot(dsmodels[i, :, 0])  # Assuming you want to plot the first dimension of your array
#         axs[plot_index].set_title(f"Plot {i+1}")  # Title for each subplot

#     # Hide any remaining empty subplots
#     for j in range(end_index - start_index, len(axs)):
#         axs[j].axis('off')

    
#     plt.tight_layout()  # Adjust layout
#     plt.savefig(f'exocomets-newtails/{start_index}-{end_index}.png',dpi=200)
#     plt.close()

In [16]:
import glob

In [19]:
files = glob.glob('../models/comets10k/*')

In [21]:
# for i in files[0:50]:
#     file = np.load(i,allow_pickle=True)
#     plt.plot(file[0],file[1])
#     plt.show()